# Analysis of Horse Race Probability Game

### Game Explaination
This game consists of 11 columns, representing numbers [2, 12], each with a specific number of pegging holes. The game is played by rolling a pair of dice and moving the corresponding horse one peg forward. The winner is the horse which crosses the finish line first. The number of holes in each column are (column, holes) [(2, 3), (3, 6), (4, 8), (5, 11), (6, 14), (7, 17), (8, 14), (9, 11), (10, 8), (11, 6), (12, 3)]

### Variation
These is a variant of the game where before the game begins, n (0, 11] numbers are rolled, and those numbers are eliminated from contention.

In [1]:
import numpy as np
import random
import seaborn as sn
import pandas as pd

In [2]:
COLUMN_HOLES = [3, 6, 8, 11, 14, 17, 14, 11, 8, 6, 3]
NUM_ELIM = 0

In [ ]:
def game(board, n):
    # Defines game play for horse game
    # Returns winner, dictionary of roll counts, and eliminated numbers
    # PARAMS:
    #   board: number of pegging holes for each column
    #   n: number of columns to be eliminated before rolling begins
    def check_winner(rolls, board, elim):
        for num, count in rolls:
            if (count >= board[num-2] and num not in elim):
                return num
        return 0
    
    elim = set() #initialize with false value to avoid empty set
    while len(elim) < n + 1:
        temp = random.randint(1, 6) + random.randint(1, 6)
        if temp not in elim:
            elim.add(temp)

    rolls = {}
    winner = 0
    while (winner == 0):
        roll = random.randint(1, 6) + random.randint(1, 6)
        if (roll not in rolls):
            rolls[roll] = 0
        rolls[roll] += 1
        winner = check_winner(rolls, board, elim)
    
    return winner, rolls, elim

In [11]:
game(COLUMN_HOLES, 0)

TypeError: cannot unpack non-iterable int object